# Tensorflow recommender systems: ranking
Code walkthrough from [tensorflow docs](https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb)

In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

/opt/homebrew/Caskroom/miniforge/base/envs/recommender_systems/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [2]:
ratings = tfds.load("movielens/100k-ratings", split="train")
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [3]:
# split into 80% train and 20% test
tf.random.set_seed(0)
shuffled = ratings.shuffle(100_000, seed=0, reshuffle_each_iteration=False)
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [4]:
# map unique movie titles and users to be able to embed it and take it back from a vector
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

2024-08-12 11:19:01.686057: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-12 11:19:02.515002: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Model arquitecture

In [5]:
class RankingModel(tf.keras.Model):
    def __init__(self,):
        super().__init__()
        embedding_dimension = 32

        # compute embeddings for a user
        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None
            ),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        # compute embeddings for movies
        self.movie_embeddings = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_movie_titles, mask_token=None
            ),
            tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
        ])

        # compute prediction
        self.ratings = tf.keras.Sequential([
            # define multiple dense layers
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            # predictions layer
            tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):
        user_id, movie_title = inputs

        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_title)
    
        return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [6]:
# test it with no trained params
RankingModel()((np.array(["42"]), np.array(["One Flew Over the Cuckoo's Nest (1975)"])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00311781]], dtype=float32)>

## Train

In [7]:
task = tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [12]:
# define model class to train and predict
class MovielensModel(tfrs.models.Model):
  def __init__(self,):
      super().__init__()
      self.ranking_model: tf.keras.Model = RankingModel()
      self.taks: tf.keras.layers.Layer = tfrs.tasks.Ranking(
          loss=tf.keras.losses.MeanSquaredError(),
          metrics=[tf.keras.metrics.RootMeanSquaredError()]
      )
  
  def call(self, features:Dict[str, tf.Tensor]) -> tf.Tensor:
      return self.ranking_model(
          (features["user_id"], features["movie_title"])
      )
  
  def compute_loss(self, features:Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
      labels = features.pop("user_rating")
      rating_predictions = self(features)
      return self.taks(labels=labels, predictions=rating_predictions)

In [ ]:
# fit process
model = MovielensModel()
model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1)
)
# shuffle batch and cache train and eval data
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=3)

In [ ]:
# evaluate
model.evaluate(cached_test, return_dict=True)

## Test Ranking model

In [ ]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
    test_ratings[movie_title] = model({
        "user_id": np.array(["42"]),
        "movie_title": np.array([movie_title])
    })

print("Ratings:")
for title, score in sorted(test_ratings, key=lambda x: x[1], reverse=True):
    print(f"{title}: {score}")